## **Práctica 1: Reglas de asociación**

Nombre: Casasola García Oscar

Nro. Cuenta: 316123747

Email: oscar.casasola.g7@gmail.com

**Objetivo**

Obtener reglas de asociación a partir de datos obtenidos de una plataforma de películas, donde los clientes pueden rentar o comprar este tipo de contenidos. 

**Características**

* Por lo general, existe un patrón en lo que ven los clientes. Por ejemplo, superhéroes en la categoría para niños.
* En este sentido, se pueden generar más ganancias, si se puede identificar la relación entre las películas. Esto es, si las películas A y B se rentan juntas, este patrón se puede aprovechar para aumentar las ganancias.
* Las personas que rentan una de estas películas pueden ser empujadas a rentar o comprar la otra, a través de campañas o sugerencias dentro de la plataforma.
* En este sentido, cada vez es común familiarizarse con los motores de recomendación en Netflix, Amazon, por nombrar los más destacados.

#### **1) Importar las bibliotecas necesarias**

In [5]:
!pip install apyori # pip es un administrador de paquetes de Python. Se instala el paquete Apyori 

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=b350ca44d49590324d4642d6f1e5be3d5ff32f90d54f640406cff84065fbf1d2
  Stored in directory: c:\users\johnc\appdata\local\pip\cache\wheels\1b\02\6c\a45230be8603bd95c0a51cd2b289aefdd860c1a100eab73661
Successfully built apyori


In [6]:
import pandas as pd                 # Para la manipulación y análisis de los datos
import numpy as np                  # Para crear vectores y matrices n dimensionales
import matplotlib.pyplot as plt     # Para la generación de gráficas a partir de los datos
from apyori import apriori

#### **2) Importar los datos**

Fuente de datos: movie_dataset.csv

In [7]:
from google.colab import files
files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
DatosMovies = pd.read_csv('movies.csv')

In [ ]:
DatosMovies

**Observaciones:**

1) Se observa que el encabezado es la primera transacción.

2) NaN indica que esa película no fue rentada o comprada en esa transacción.

In [ ]:
DatosMovies = pd.read_csv('movies.csv', header=None)
DatosMovies.head(6)

#### **3) Procesamiento de los datos**

**Exploración**

Antes de ejecutar el algoritmo es recomendable observar la distribución de la frecuencia de los elementos.

In [ ]:
#Se incluyen todas las transacciones en una sola lista
Transacciones = DatosMovies.values.reshape(-1).tolist() #-1 significa 'dimensión desconocida'

#Se crea una matriz (dataframe) usando la lista y se incluye una columna 'Frecuencia'
Lista = pd.DataFrame(Transacciones)
Lista['Frecuencia'] = 1

#Se agrupa los elementos
Lista = Lista.groupby(by=[0], as_index=False).count().sort_values(by=['Frecuencia'], ascending=True) #Conteo
Lista['Porcentaje'] = (Lista['Frecuencia'] / Lista['Frecuencia'].sum()) #Porcentaje
Lista = Lista.rename(columns={0 : 'Item'})

#Se muestra la lista
Lista

In [ ]:
# Se genera un gráfico de barras
plt.figure(figsize=(16,20), dpi=300)
plt.ylabel('Item')
plt.xlabel('Frecuencia')
plt.barh(Lista['Item'], width=Lista['Frecuencia'], color='blue')
plt.show()

**Preparación**

La función Apriori de Python requiere que el conjunto de datos tenga la forma de una lista de listas, donde cada transacción es una lista interna dentro de una gran lista. 

Los datos actuales están en un dataframe de Pandas, por lo que, se requiere convertir en una lista.

In [ ]:
#Se crea una lista de listas a partir del dataframe y se remueven los 'NaN'
#level=0 especifica desde el primer índice
MoviesLista = DatosMovies.stack().groupby(level=0).apply(list).tolist()
MoviesLista 

In [ ]:
#MoviesLista = []
#for i in range(0, 7459):
#  MoviesLista.append([str(DatosMovies.values[i,j]) for j in range(0, 20)])

#NuevaLista = []
#for item in MoviesLista:
#  if str(item) != 'nan':
#    NuevaLista.append(item)
#print(NuevaLista)

#### **4) Aplicación del algoritmo**

**Configuración 1**

Obtener reglas para aquellas películas que se hayan rentado al menos 10 veces en un día (70 veces en una semana):

i) El soporte mínimo se calcula de 70/7460 = 0.00938 (1%).

ii) La confianza mínima para las reglas de 30%

iii) La elevación de 2.

**Observación:** Estos valores se eligen arbitrariamente, por lo que, se recomienda probar valores y analizar la diferencia en las reglas.

In [ ]:
ReglasC1 = apriori(MoviesLista, 
                   min_support=0.01, 
                   min_confidence=0.3, 
                   min_lift=2)

Se convierte las reglas encontradas por la clase apriori en una lista, puesto que es más fácil ver los resultados.

In [ ]:
ResultadosC1 = list(ReglasC1)
print(len(ResultadosC1)) #Total de reglas encontradas 

In [ ]:
ResultadosC1

In [ ]:
pd.DataFrame(ResultadosC1)

Son 9 reglas. A manera de ejemplo se imprime la primera regla:

In [ ]:
print(ResultadosC1[0])

La primera regla contiene dos elementos: **'Kung Fu Panda'** y **'Jumanji'** que se vieron juntos.

* Esto tiene sentido, las personas que ven películas familiares, en este caso de corte infantil, suelen ver también más películas del mismo tipo, como Kung Fu Panda (2016) y Jumanji (2017).

* El soporte es de 0.016 (1.6%), la confianza de 0.32 (32%) y elevación de 3.27, esto representa que existe 3 veces más probabilidades de que los que vean Kung Fu Panda miren también Jumanji, o viceversa.

In [ ]:
print(ResultadosC1[1])
print(ResultadosC1[2])

In [ ]:
for item in ResultadosC1:
  #El primer índice de la lista
  Emparejar = item[0]
  items = [x for x in Emparejar]
  print("Regla: " + str(item[0]))

  #El segundo índice de la lista
  print("Soporte: " + str(item[1]))

  #El tercer índice de la lista
  print("Confianza: " + str(item[2][0][2]))
  print("Lift: " + str(item[2][0][3])) 
  print("=====================================") 

**Configuración 2**

Obtener reglas para aquellas películas que se hayan visto al menos 210 veces a la semana (30 por día):

i) El soporte mínimo se calcula de 210/7460 = 0.028 (2.8%).

ii) La confianza mínima para las reglas de 30%.

iii) La elevación mayor a 1.

In [ ]:
ReglasC2 = apriori(MoviesLista, 
                   min_support=0.028, 
                   min_confidence=0.3, 
                   min_lift = 1.01)

In [ ]:
ResultadosC2 = list(ReglasC2)
print(len(ResultadosC2))

In [ ]:
ResultadosC2 

In [ ]:
pd.DataFrame(ResultadosC2)

In [ ]:
print(ResultadosC2[0])

La primera regla contiene dos elementos: **'Get Out'** y **'Beirut'** que se vieron juntos.

Esto también tiene sentido, las personas que ven películas de espionaje, como Beirut (2018), tienen gustos afines con películas de terror, como Get Out (2017).

El soporte es de 0.028 (2.8%), la confianza de 0.33 (33%) y una elevación de 1.83, esto representa que existe casi 2 veces más probabilidades de que los que vean Get Out miren también Beirut, o viceversa.

In [ ]:
print(ResultadosC2[1])
print(ResultadosC2[2])

In [ ]:
for item in ResultadosC2:
  #El primer índice de la lista
  Emparejar = item[0]
  items = [x for x in Emparejar]
  print("Regla: " + str(item[0]))

  #El segundo índice de la lista
  print("Soporte: " + str(item[1]))

  #El tercer índice de la lista
  print("Confianza: " + str(item[2][0][2]))
  print("Lift: " + str(item[2][0][3])) 
  print("=====================================") 